# This colab
- Takes the Dataframe processed earlier
- embeddes 'product-description" into Chroma Vector DB
- Adds the following as metadata
- Adds the image ids s id

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [1]:
!pip install salesforce-lavis

# Install Necessay stuff

In [2]:
!pip install transformers==4.26.1
!pip install langchain_community
!pip install chromadb
!pip install bitsandbytes

# Ref: https://python.langchain.com/docs/integrations/vectorstores/chroma/
#!pip install -qU langchain-huggingface

!pip install -qU "langchain-chroma>=0.1.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.24
    Uninstalling langchain-core-0.3.24:
      Successfully uninstalled langchain-core-0.3.24
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.2
    Uninstalling langchain-text-splitters-0.3.2:
      Successfully uninstalled langchain-text-splitters-0.3.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.11
    Uninstalling langchain-0.3.11:
      Successfully uninstalled langchain-0.3.11
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependenci

In [3]:
# mount storage
from google.colab import drive
drive.mount('/content/drive')
#%cd "/gdrive/MyDrive/Interview Kickstart/MLSwitchup/Capstone/ShopTalk"
%cd /content/drive/MyDrive/ik-ml/capstone/github/ShopTalk/
%ls

Mounted at /content/drive
/content/drive/MyDrive/ik-ml/capstone/github/ShopTalk
 1.0.2                         assets/                  code/          saved-data/
'Agent workflow sketch.pptx'  'BLIP-2 Fine-Tune.docx'   python_envs/   ShopTalk-blobs@


In [ ]:
%ls /content/drive/MyDrive/ik-ml/capstone/github/ShopTalk/ShopTalk-blobs/embeddings/

embeddings_abo.pkl  embeddings_gs.pkl  embeddings_pretrain.pkl


In [ ]:
%ls /content/drive/MyDrive/ik-ml/capstone/github/ShopTalk/assets

blip-2-processors.pkl  language_detector.tflite  sofa.jpg


In [4]:
import pickle
import torch
from lavis.models import load_model_and_preprocess

from pydantic import BaseModel
#import base64
#from io import BytesIO
#from PIL import Image


/usr/local/lib/python3.10/dist-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/usr/local/lib/python3.10/dist-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


In [ ]:
%pwd


'/content/drive/MyDrive/ik-ml/capstone/github/ShopTalk'

In [43]:
blip2_file_path="/content/drive/MyDrive/ik-ml/capstone/github/ShopTalk/assets/blip-2-processors.pkl"
# the (eval) preprocessors for the model
with open(blip2_file_path, 'rb') as f:
    vis_processor, text_processor = pickle.load(f)

In [44]:
# the model itself
print('Loading model')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#model = torch.load("model-saves/pretrain_1epoch.pt")
#model = torch.load("/content/drive/MyDrive/ik-ml/capstone/github/ShopTalk/ShopTalk-blobs/model_files/blip-2-gs-trained-1epoch.pt")
# torch.load with map_location=torch.device('cpu')
model=torch.load("/content/drive/MyDrive/ik-ml/capstone/github/ShopTalk/ShopTalk-blobs/model_files/blip-2-gs-trained-1epoch.pt", map_location=torch.device('cpu'))
model.to(device)
model.eval()
print('Done loading model')

Loading model


<ipython-input-44-9eaf98dbb114>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model=torch.load("/content/drive/MyDrive/ik-ml/capstone/github/ShopTalk/ShopTalk-blobs/model

Done loading model


In [45]:
#from chromadb.utils import embedding_functions
#from chromadb import Documents, EmbeddingFunction, Embeddings

#from chromadb.utils import embedding_functions
from chromadb import Documents, EmbeddingFunction, Embeddings

class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, text_processor, model):
      super().__init__()
      self.text_processor = text_processor
      self.model = model

    def embed_documents(self, texts):
        sample = {'text_input': texts}
        embedding = self.model.extract_features(sample, mode='text').text_embeds[0,0,:].detach().cpu().tolist()
        return [embedding]

    def embed_query(self, query: str):
        sample = {'text_input': [query]}
        embedding = self.model.extract_features(sample, mode='text').text_embeds[0,0,:].detach().cpu().tolist()
        return embedding


In [46]:
d = ['this is sample test est']
myembed=MyEmbeddingFunction(text_processor, model)
d_embed=myembed.embed_query('This test')
print(len(d_embed))
embed02=myembed.embed_documents(d)
print(embed02)


768
[[-0.079023577272892, 0.2892645299434662, -0.05367223173379898, 0.473632276058197, 0.24190695583820343, -0.2705511152744293, 0.22005359828472137, -0.07970192283391953, 0.23333840072155, -0.37240347266197205, 0.6528847217559814, -0.18658335506916046, 0.12403398752212524, -0.038528259843587875, -0.3905896842479706, 0.6771913766860962, 0.6537386775016785, -0.029794683679938316, 0.2737199664115906, -0.26839128136634827, -0.09419342130422592, -0.17069073021411896, -0.06763269752264023, -0.274308979511261, 0.2064734548330307, -0.515960693359375, 0.5667680501937866, -0.14930438995361328, 0.08230314403772354, 0.27935853600502014, -0.5972844958305359, -0.07579170912504196, 0.005694256164133549, 0.024350928142666817, 0.15625882148742676, 0.4202483594417572, -0.3529123365879059, 0.11153369396924973, 0.18387559056282043, 0.30305561423301697, -0.27965086698532104, -0.6220105886459351, 0.16754917800426483, -0.30523425340652466, 0.11020464450120926, 0.05390762537717819, -0.5983800292015076, 0.402

In [48]:
import pandas as pd
# from langchain_huggingface import HuggingFaceEmbeddings
#from langchain_chroma import Chroma
from langchain_chroma import Chroma
from tqdm import tqdm
#from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import numpy as np # Import numpy


class VectorStore ():
  def __init__(self):
    self.preprocessed_data = None
    self.preprocessed_data_file = None
    self.content_col_name = None
    self.metadata_columns = None
    self.vector_store = None
    self.collection_name = None
    self.db_file_path = None
    self.embeddings = None
    self.retriever = None
    self.mdf = None


  # Takes the preprocessed dataframe saved as pickle file
  def loadPreprocessedData(self, pre_processed_data_file):
    self.preprocessed_data_file = pre_processed_data_file
    self.preprocessed_data = pd.read_pickle(self.preprocessed_data_file)
    #self.preprocessed_data.dropna(subset=[content_col_name], inplace=True)
    return True

  def getData(self):
    return self.mdf

  def getPreprocessedData(self):
    return self.preprocessed_data

  # if the data file exists, DB will be created from the file. if not, new DB is created
  def createVectorStore(self, collection_name, db_file_path, embeddings):
    self.collection_name = collection_name
    self.db_file_path = db_file_path
    self.embeddings = embeddings

    self.vector_store = Chroma(collection_name=self.collection_name, embedding_function=self.embeddings,
                        persist_directory=self.db_file_path,  # Where to save data locally, remove if not necessary
                    )
    return True


  def get_row_count(self):
    return self.vector_store.count()


  def getVectorStore(self):
    return self.vector_store

  def format_selected_columns(self, df, excluded_columns, existing_column_name):
    """
    Formats all columns except those specified in excluded_columns into "<column name> : <value>" text
    and appends to an existing column.

    Args:
      df: The input DataFrame.
      excluded_columns: A list of column names to exclude from formatting.
      existing_column_name: The name of the existing column to append the formatted text to.

    Returns:
      The DataFrame with the existing column modified.
    """

    # Get all columns except those in excluded_columns
    selected_columns = [col for col in df.columns if col not in excluded_columns]

    # Loop through rows and format selected columns, appending to existing column
    for index, row in df.iterrows():
      formatted_text = ""
      for col in selected_columns:
        formatted_text += f"{col} : {row[col]} "  # Format text

      # Ensure the existing value is a string before appending
      current_value = df.loc[index, existing_column_name]
      df.loc[index, existing_column_name] = str(current_value) + " " + formatted_text.strip()  # Append formatted text to existing column, ensuring string type
      #df.loc[index, existing_column_name] += " " + formatted_text.strip()  # Append formatted text to existing column

    return df



  #db = Chroma.from_documents(texts, embeddings)

  def add_data_to_vector_store(self, test_count):
    total_count = self.preprocessed_data.shape[0]
    print('Total Records to add {}'.format(total_count))

    if ( test_count == 0 ):
      test_run = False
    else:
      print('This is test run processing only {} rows.'.format(test_count))
      test_run = True

    pcount = 0
    skipped_count = 0
    for index, row in enumerate(tqdm(self.preprocessed_data.itertuples(), total=total_count)):
      try:
        pcount = pcount +1
        row_dict = row._asdict()

        self.vector_store.add_texts(
            texts=[""],
            embeddings=row_dict.get('embedding'),
            collection_name=self.collection_name,
            ids=[row_dict.get('item_id')]
        )

      except Exception as e:
        skipped_count = skipped_count + 1
        print('--------------- Exception ------------------')
        print(e)
        print('---------------------------------------------')

      if ( pcount > test_count and test_run):
        #print('count {} and test_run {}'.format(pcount, test_run))
        break

    print('Skipped {} rows out of total rows {}'.format(skipped_count, pcount))
    return True


  def addDataToVectorStoreIteratively(self, content_col_name, metadata_cols_list):
      self.content_col_name = content_col_name
      self.metadata_columns = metadata_cols_list

      total_count = self.preprocessed_data.shape[0]
      print('Total Records to add {}'.format(total_count))

      # convert the metadata columns
      #mdf = self.preprocessed_data[self.metadata_columns]
      count = 0
      skipped_count = 0
      for index, row in enumerate(tqdm(self.preprocessed_data.itertuples(), total=total_count)):
        try:
          count = count +1
          # Convert the namedtuple 'row' to a dictionary
          row_dict = row._asdict()
          print('Processing row: {}'.format(count))
          mds=self.row_to_dict(row_dict, metadata_cols_list)
          idlist=[row_dict['Index']]
          text_to_add = row_dict[self.content_col_name]

          if ( text_to_add == '' or text_to_add == []):
            skipped_count = skipped_count + 1
            print(' -> Skipping row with index {} as the text to add is empty'.format(row_dict['Index']))
            continue

          text_list = [text_to_add]

          self.vector_store.add_texts(
              embedding_function=self.embeddings,
              collection_name=self.collection_name,
              ids=idlist, # Access 'Index' using dictionary key
              texts = text_list, # Access content column using dictionary key
              metadatas=mds  # Pass dictionary to row_to_dict
          )

        except Exception as e:
          print('--------------- Exception ------------------')
          print(e)
          print('---------------------------------------------')

      print('Out of Total records {}, {} are skipped due to data issues'.format(count, skipped_count))
      return True

  def row_to_dict(self, row, column_names):
      """Converts a DataFrame row (now a dictionary) to a dictionary with specified columns and null value handling.
      """

      dict_list = []
      for column_name in column_names:
          value = row.get(column_name) # Use get() for dictionary access
          row_dict = {}
          # Check if value is a list or a NumPy array
          if isinstance(value, (list, np.ndarray)):
              # If it's a list or array, convert NaN values to empty strings within the list/array
              value = ["" if pd.isnull(x) else x for x in value]
          else:
              # If it's a single value, check for NaN and replace with an empty string
              if pd.isnull(value):
                  value = ''
          row_dict[column_name] = value

      dict_list.append(row_dict)
      return dict_list


  def getRetriever(self):
    if ( self.retriever is None):
      self.retriever = self.vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1, "fetch_k": 5})
    return self.retriever

  def addDataToVectorStoreBulk(self, content_col_name, metadata_cols_list):
    self.content_col_name = content_col_name
    self.metadata_columns = metadata_cols_list

    # convert the metadata columns
    mdf = self.preprocessed_data[self.metadata_columns]
    # Convert the product descriptions to strings before adding to ChromaDB
    # The tolist() method converts the Pandas Series into a list for ChromaDB
    self.vector_store.add_texts(
        embedding_function=self.embeddings,
        collection_name=self.collection_name,
        # if the df indexes are to be used as ids.if column content, then this requires change
        ids=self.preprocessed_data.index.tolist(),
        texts=self.preprocessed_data[content_col_name].astype(str).tolist(), # Convert to strings
        metadatas=mdf[metadata_cols_list].to_dict(orient="records") ,
    )
    return True



In [ ]:

# Test the class with dummay data
vs = VectorStore()
vs.loadPreprocessedData("/content/drive/MyDrive/ik-ml/capstone/github/ShopTalk/ShopTalk-blobs/embeddings/embeddings_pretrain.pkl")
data = vs.getPreprocessedData()


In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 571196 entries, 0 to 571195
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   image_id   571060 non-null  object
 1   item_id    571196 non-null  object
 2   embedding  571196 non-null  object
dtypes: object(3)
memory usage: 13.1+ MB


In [42]:
vs.add_data_to_vector_store( 2500)

Total Records to add 571196
This is test run processing only 2500 rows.


  0%|          | 286/571196 [00:00<03:52, 2457.15it/s]

--------------- Exception ------------------
'NoneType' object has no attribute 'add_texts'
---------------------------------------------
--------------- Exception ------------------
'NoneType' object has no attribute 'add_texts'
---------------------------------------------
--------------- Exception ------------------
'NoneType' object has no attribute 'add_texts'
---------------------------------------------
--------------- Exception ------------------
'NoneType' object has no attribute 'add_texts'
---------------------------------------------
--------------- Exception ------------------
'NoneType' object has no attribute 'add_texts'
---------------------------------------------
--------------- Exception ------------------
'NoneType' object has no attribute 'add_texts'
---------------------------------------------
--------------- Exception ------------------
'NoneType' object has no attribute 'add_texts'
---------------------------------------------
--------------- Exception --------

  0%|          | 1136/571196 [00:00<02:37, 3627.12it/s]


--------------- Exception ------------------
'NoneType' object has no attribute 'add_texts'
---------------------------------------------
--------------- Exception ------------------
'NoneType' object has no attribute 'add_texts'
---------------------------------------------
--------------- Exception ------------------
'NoneType' object has no attribute 'add_texts'
---------------------------------------------
--------------- Exception ------------------
'NoneType' object has no attribute 'add_texts'
---------------------------------------------
--------------- Exception ------------------
'NoneType' object has no attribute 'add_texts'
---------------------------------------------
--------------- Exception ------------------
'NoneType' object has no attribute 'add_texts'
---------------------------------------------
--------------- Exception ------------------
'NoneType' object has no attribute 'add_texts'
---------------------------------------------
--------------- Exception -------

True

  0%|          | 2500/571196 [00:00<01:53, 5015.22it/s]

In [31]:
retriever = vs.getRetriever()

AttributeError: 'Chroma' object has no attribute 'getRetriever'

AttributeError: 'VectorStoreRetriever' object has no attribute 'count'

In [15]:
retriever.invoke("Give me all the embeddings data")
#retriever.invoke("Give me white color shoes list", filter={"source": "news"})

[Document(metadata={}, page_content='')]

In [16]:
retriever.invoke("give me list of all content")

[Document(metadata={}, page_content='')]